In [ ]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, lpSum
import tkinter as tk
from tkinter import messagebox
from datetime import datetime, timedelta
import customtkinter as ctk

# Load data
car_data = pd.read_csv("TATA_Production.csv")

# Fixed values
DAILY_LABOR_HOURS = 54000  # 6000 workers, 9 hours/day, two shifts
DAILY_MACHINE_HOURS = 30000  # 55% of labor hours
MAX_CARS_PER_DAY = 400  # Plant capacity

# Global demand table
demand_table = {}

HISTORY_FILE = "production_history.csv"

# CustomTkinter setup
ctk.set_appearance_mode("dark")  
ctk.set_default_color_theme("blue")  

class CarProductionApp(ctk.CTk):
    def __init__(self):
        super().__init__()
        self.title("Car Production Optimization")
        self.attributes('-fullscreen', True)  
        self.grid_columnconfigure(0, weight=1)

        # Exit Button (Top-Right Corner)
        self.exit_button = ctk.CTkButton(
            self, text="X", font=("Arial", 16, "bold"), width=40, fg_color="red", hover_color="darkred",
            command=self.destroy
        )
        self.exit_button.place(x=self.winfo_screenwidth() - 50, y=10)  
        self.exit_button.lift()  

        # Main Frame 
        self.main_frame = ctk.CTkScrollableFrame(self, width=1000, height=self.winfo_screenheight() - 50)
        self.main_frame.grid(row=0, column=0, padx=20, pady=20, sticky="nsew")
        self.main_frame.grid_columnconfigure(0, weight=1)

        # Input Section 
        self.input_frame = ctk.CTkFrame(self.main_frame)
        self.input_frame.grid(row=0, column=0, padx=10, pady=10, sticky="n")
        self.input_frame.grid_columnconfigure(0, weight=1)

        # Title Label 
        self.title_label = ctk.CTkLabel(
            self.input_frame, text="Optimize Car Production", font=("Arial", 24, "bold"),
            fg_color="blue", corner_radius=10, padx=20, pady=10
        )
        self.title_label.grid(row=0, column=0, columnspan=2, pady=20)

        # Raw Material Input
        self.create_label_entry(self.input_frame, "Total Raw Material (kg):", 1)
        self.total_raw_material_var = self.create_entry(self.input_frame, 1)

        # Completion Date Input
        self.create_label_entry(self.input_frame, "Completion Date (YYYY-MM-DD):", 2)
        self.completion_time_var = self.create_entry(self.input_frame, 2)

        # Car Model Dropdown
        self.create_label_entry(self.input_frame, "Car Model:", 3)
        self.selected_car_var = ctk.CTkComboBox(
            self.input_frame, values=list(car_data["Product Name"]), width=200, font=("Arial", 16)
        )
        self.selected_car_var.grid(row=3, column=1, padx=10, pady=10, sticky="ew")

        # Demand Input
        self.create_label_entry(self.input_frame, "Demand:", 4)
        self.demand_var = self.create_entry(self.input_frame, 4)

        # Add Demand Button 
        self.add_demand_button = ctk.CTkButton(
            self.input_frame, text="Add Demand", command=self.add_demand, font=("Arial", 16), width=120, height=30
        )
        self.add_demand_button.grid(row=5, column=0, columnspan=2, padx=10, pady=10, sticky="ew")

        # Demand Display Section
        self.demand_display_text = ctk.CTkLabel(
            self.input_frame, text="", font=("Courier New", 14), justify="left"
        )
        self.demand_display_text.grid(row=7, column=0, columnspan=2, padx=10, pady=10, sticky="nsew")
        self.update_demand_display()

        # Optimize Button 
        self.optimize_button = ctk.CTkButton(
            self.input_frame, text="Optimize Production", command=self.optimize_production, font=("Arial", 16), width=150, height=30
        )
        self.optimize_button.grid(row=8, column=0, columnspan=2, padx=10, pady=20, sticky="ew")

        # Output Display 
        self.result_text = ctk.CTkLabel(
            self.main_frame, text="", font=("Courier New", 16, "bold"),
            justify="center", width=1000, height=300
        )
        self.result_text.grid(row=1, column=0, padx=20, pady=20, sticky="s")

    def create_label_entry(self, parent, text, row):
        """Creates a label and places it in the given row."""
        label = ctk.CTkLabel(parent, text=text, font=("Arial", 16))
        label.grid(row=row, column=0, padx=10, pady=10, sticky="e")

    def create_entry(self, parent, row):
        """Creates an entry field and places it in the given row."""
        entry = ctk.CTkEntry(parent, width=200, font=("Arial", 16))
        entry.grid(row=row, column=1, padx=10, pady=10, sticky="w")
        return entry

    def add_demand(self):
        """Adds demand for a specific car model to the demand table."""
        try:
            car = self.selected_car_var.get()
            demand = int(self.demand_var.get())

            if not car:
                raise ValueError("Please select a car model.")
            if demand <= 0:
                raise ValueError("Demand must be a positive number.")

            # Add the demand for the selected car
            demand_table[car] = demand_table.get(car, 0) + demand

            # Update the demand display
            self.update_demand_display()
            self.selected_car_var.set("")  
            self.demand_var.delete(0, tk.END)  

        except ValueError as ve:
            messagebox.showerror("Invalid Input", str(ve))
        except Exception as e:
            messagebox.showerror("Error", f"An unexpected error occurred: {e}")

    def optimize_production(self):
        """Optimizes the production schedule using LP for each day until the completion date."""
        try:
            # inputs
            completion_date = datetime.strptime(self.completion_time_var.get(), "%Y-%m-%d")
            start_date = datetime.today()
            total_raw_material_available = float(self.total_raw_material_var.get())

            if not demand_table:
                raise ValueError("No demand entered.")
            if total_raw_material_available <= 0:
                raise ValueError("Raw material must be positive.")

            # Initialize tracking variables
            remaining_demand = {car: qty for car, qty in demand_table.items()}
            production_schedule = []
            total_labor_used = 0
            total_machine_used = 0
            total_raw_used = 0
            insufficient_material = {}

            current_date = start_date

            # Optimize each day until completion date or demand is met
            while current_date <= completion_date and remaining_demand and total_raw_material_available > 0:
                # Sort remaining demand to prioritize higher demand
                sorted_demand = dict(sorted(remaining_demand.items(), key=lambda item: item[1], reverse=True))

                # Create LP problem for the day
                model = LpProblem("Daily_Production", LpMaximize)
                production_vars = LpVariable.dicts("Produce", sorted_demand.keys(), 0, None, cat='Integer')

                # Objective Function
                model += lpSum(production_vars[car] for car in sorted_demand)

                # Constraints
                labor_constraint = lpSum(production_vars[car] * car_data.loc[car_data["Product Name"] == car, "Labor Hours per Unit"].values[0] for car in sorted_demand) <= DAILY_LABOR_HOURS
                machine_constraint = lpSum(production_vars[car] * car_data.loc[car_data["Product Name"] == car, "Machine Hours per Unit"].values[0] for car in sorted_demand) <= DAILY_MACHINE_HOURS
                material_constraint = lpSum(production_vars[car] * car_data.loc[car_data["Product Name"] == car, "Raw Material per Unit (kg)"].values[0] for car in sorted_demand) <= total_raw_material_available
                capacity_constraint = lpSum(production_vars[car] for car in sorted_demand) <= MAX_CARS_PER_DAY

                model += labor_constraint
                model += machine_constraint
                model += material_constraint
                model += capacity_constraint

                # Demand constraints
                for car in sorted_demand:
                    model += production_vars[car] <= sorted_demand[car]

                model.solve()

                # Extract production results
                daily_production = {}
                for car in sorted_demand:
                    produced = int(production_vars[car].varValue)
                    if produced > 0:
                        daily_production[car] = produced

                for car, qty in daily_production.items():
                    # Update remaining demand
                    remaining_demand[car] -= qty
                    if remaining_demand[car] == 0:
                        del remaining_demand[car]

                    # Update resources used
                    labor = qty * car_data.loc[car_data["Product Name"] == car, "Labor Hours per Unit"].values[0]
                    machine = qty * car_data.loc[car_data["Product Name"] == car, "Machine Hours per Unit"].values[0]
                    raw = qty * car_data.loc[car_data["Product Name"] == car, "Raw Material per Unit (kg)"].values[0]

                    total_labor_used += labor
                    total_machine_used += machine
                    total_raw_used += raw
                    total_raw_material_available -= raw

                    # Record production
                    production_schedule.append([car, current_date.strftime("%Y-%m-%d"), qty])

                current_date += timedelta(days=1)

            # Save history
            self.save_production_history(production_schedule)

            # Generate result summary
            resource_usage = {}
            for car in demand_table:
                initial_demand = demand_table[car]
                produced = initial_demand - remaining_demand.get(car, 0)
                completion_dates = [entry[1] for entry in production_schedule if entry[0] == car]
                completion_day = max(completion_dates) if completion_dates else "Not Met"

                labor_per_unit = car_data.loc[car_data["Product Name"] == car, "Labor Hours per Unit"].values[0]
                machine_per_unit = car_data.loc[car_data["Product Name"] == car, "Machine Hours per Unit"].values[0]
                raw_per_unit = car_data.loc[car_data["Product Name"] == car, "Raw Material per Unit (kg)"].values[0]

                resource_usage[car] = {
                    "Labor Hours": produced * labor_per_unit,
                    "Machine Hours": produced * machine_per_unit,
                    "Raw Material Used": produced * raw_per_unit,
                    "Total Produced": produced,
                    "Completion Date": completion_day
                }

                # Track insufficient material
                if remaining_demand.get(car, 0) > 0:
                    needed = remaining_demand[car] * raw_per_unit
                    insufficient_material[car] = needed

            # Format results
            result = "\nProduction Schedule:\n"
            result += "+-------------------+-------------------+---------------+\n"
            result += "| Date              | Car Model         | Daily Units   |\n"
            result += "+-------------------+-------------------+---------------+\n"
            for entry in production_schedule:
                result += f"| {entry[1]:<17} | {entry[0]:<17} | {entry[2]:<13} |\n"
            result += "+-------------------+-------------------+---------------+\n"

            result += "\nResource Usage by Model:\n"
            result += "+-------------------+---------------+----------------+----------------+-------------------+-------------------+\n"
            result += "| Car Model         | Labor Hours   | Machine Hours  | Raw Material   | Total Produced    | Completion Date   |\n"
            result += "+-------------------+---------------+----------------+----------------+-------------------+-------------------+\n"
            for car, usage in resource_usage.items():
                result += f"| {car:<17} | {usage['Labor Hours']:<13.2f} | {usage['Machine Hours']:<14.2f} | {usage['Raw Material Used']:<14.2f} | {usage['Total Produced']:<17} | {usage['Completion Date']:<17} |\n"
            result += "+-------------------+---------------+----------------+----------------+-------------------+-------------------+\n"

            result += f"\nTotal Labor Hours Used: {total_labor_used:.2f}"
            result += f"\nTotal Machine Hours Used: {total_machine_used:.2f}"
            result += f"\nTotal Raw Material Used: {total_raw_used:.2f}"
            result += f"\nRemaining Raw Material: {total_raw_material_available:.2f} kg"

            if insufficient_material:
                result += "\n\nInsufficient Raw Material:\n"
                for car, needed in insufficient_material.items():
                    result += f"{car}: Additional {needed:.2f} kg required.\n"

            self.result_text.configure(text=result)

        except ValueError as ve:
            messagebox.showerror("Invalid Input", str(ve))
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {e}")

    def update_demand_display(self):
        """Updates the displayed demand table in the GUI."""
        display = "\nCurrent Demand:\n"
        display += "+----------------------+----------+\n"
        display += "| Car Model           | Demand   |\n"
        display += "+----------------------+----------+\n"
        for car, qty in demand_table.items():
            display += f"| {car:<20} | {qty:<8} |\n"
        display += "+----------------------+----------+\n"
        self.demand_display_text.configure(text=display)

    def save_production_history(self, production_schedule):
        """Saves the production schedule to a CSV file."""
        df = pd.DataFrame(production_schedule, columns=["Car Model", "Date", "Units Produced"])
        try:
            existing_df = pd.read_csv(HISTORY_FILE)
            df = pd.concat([existing_df, df], ignore_index=True)
        except FileNotFoundError:
            pass
        df.to_csv(HISTORY_FILE, index=False)

# Run app
if __name__ == "__main__":
    app = CarProductionApp()
    app.mainloop()